In [1]:
#Model for recognizing and extracting plate numbers 

In [1]:
# Install OpenCV, PyTorch (if not installed), and pytesseract.
!pip --default-timeout=100 install opencv-python-headless pytesseract
import sys
!{sys.executable} -m pip install opencv-python-headless
!{sys.executable} -m pip install pytesseract
!{sys.executable} -m pip install ultralytics --upgrade
!{sys.executable} -m pip install numpy==1.26.0
!pip install easyocr

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached ultralytics-8.3.129-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.129-py3-none-any.whl (1.0 MB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)

   ---------------------------------------- 0/3 [opencv-python]
   ---------------------------------------- 0/3 [opencv-python]
   ---------------------------------------- 0/3 [opencv-python]
   ---------------------------------------- 0/3 [opencv-python]
   -------------

In [1]:
!git config --global http.postBuffer 524288000
# Clone the YOLOv5 repository.
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
#check if yolov5 has been cloned successfully
!dir yolov5  # Window

 Volume in drive C is Windows 11
 Volume Serial Number is 6A75-E4F2

 Directory of C:\Users\user\Desktop\yolov5

05/08/2025  04:49 PM    <DIR>          .
05/08/2025  06:08 PM    <DIR>          ..
05/08/2025  03:30 PM             3,923 .dockerignore
05/08/2025  03:30 PM                77 .gitattributes
05/08/2025  03:30 PM    <DIR>          .github
05/08/2025  03:30 PM             4,268 .gitignore
05/08/2025  03:30 PM            14,440 benchmarks.py
05/08/2025  03:30 PM               407 CITATION.cff
05/08/2025  03:30 PM    <DIR>          classify
05/08/2025  03:30 PM             5,506 CONTRIBUTING.md
05/08/2025  03:30 PM    <DIR>          data
05/08/2025  03:30 PM            24,199 detect.py
05/08/2025  03:30 PM            70,123 export.py
05/08/2025  03:30 PM            24,507 hubconf.py
05/08/2025  03:30 PM            35,184 LICENSE
05/08/2025  04:49 PM    <DIR>          models
05/08/2025  03:30 PM             5,550 pyproject.toml
05/08/2025  03:30 PM            50,512 README.md
05/0

File Not Found


In [3]:
import os
import shutil
import random
import glob

# Define paths for the original VOC dataset.
voc_path = r'C:\Users\user\Desktop\VOC-LICENSE-PLATE'
images_source = os.path.join(voc_path, 'JPEGImages')
annos_source = os.path.join(voc_path, 'Annotations')

# Create new directory structure:
# data/
#   images/train, images/val
#   labels/train, labels/val
os.makedirs('data/images/train', exist_ok=True)
os.makedirs('data/images/val', exist_ok=True)
os.makedirs('data/labels/train', exist_ok=True)
os.makedirs('data/labels/val', exist_ok=True)

# Get all image files (assuming .jpg; extend the tuple if you have png, etc.)
image_files = glob.glob(os.path.join(images_source, '*.png'))
image_files.sort()   # sort for consistency

# Shuffle and split (80% train, 20% validation)
random.seed(42)
random.shuffle(image_files)
split_index = int(0.8 * len(image_files))
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Copy image files into the new structure.
def copy_files(file_list, dest_dir):
    for file_path in file_list:
        shutil.copy(file_path, dest_dir)

copy_files(train_files, 'data/images/train')
copy_files(val_files, 'data/images/val')

print(f"Copied {len(train_files)} images to data/images/train")
print(f"Copied {len(val_files)} images to data/images/val")


Copied 346 images to data/images/train
Copied 87 images to data/images/val


In [4]:
import cv2
import xml.etree.ElementTree as ET

def convert_annotation(xml_file, img_width, img_height):
    """
    Parses a VOC xml file and returns the annotation in YOLO format.
    Assumes one class (license_plate) with id = 0.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_lines = []
    
    for obj in root.iter('object'):
        # Get class name (if needed, you could check that it's 'license_plate')
        class_name = obj.find('name').text
        class_id = 0  # since we have one class
        
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        # Calculate normalized coordinates
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        w = (xmax - xmin) / img_width
        h = (ymax - ymin) / img_height
        
        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
    return "\n".join(yolo_lines)

# Function to process a list of images and convert corresponding annotations.
def process_annotations(image_list, split='train'):
    for image_path in image_list:
        # Get the base filename without extension.
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        xml_path = os.path.join(annos_source, base_name + '.xml')
        if not os.path.exists(xml_path):
            print(f"Warning: Annotation file not found for {image_path}")
            continue
        
        # Read image to get dimensions.
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read {image_path}")
            continue
        img_height, img_width = img.shape[:2]
        
        # Convert annotation to YOLO format.
        yolo_annotation = convert_annotation(xml_path, img_width, img_height)
        
        # Define destination label file.
        label_file = os.path.join('data/labels', split, base_name + '.txt')
        with open(label_file, 'w') as f:
            f.write(yolo_annotation)

# Process training and validation images.
process_annotations(train_files, split='train')
process_annotations(val_files, split='val')
print("Annotation conversion complete.")


Annotation conversion complete.


In [5]:
%%writefile license_plate.yaml
# YOLOv5 License Plate Dataset Configuration
path: data  # root folder containing "images" and "labels"
train: images/train
val: images/val

nc: 1
names: ['license_plate']



Overwriting license_plate.yaml


In [12]:
# Change directory to yolov5 repository if needed.
%cd yolov5

# Start training using YOLOv5.
# Make sure that the "license_plate.yaml" file is in the correct location (or provide the full path).
!python train.py --img 640 --batch 8 --epochs 50 --data "C:\Users\user\Desktop\license_plate.yaml" --weights yolov5s.pt



[WinError 2] The system cannot find the file specified: 'yolov5'
C:\Users\user\Desktop\yolov5\yolov5
Command 'git fetch origin' timed out after 5 seconds


train: weights=yolov5s.pt, cfg=, data=C:\Users\user\Desktop\license_plate.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
YOLOv5  v7.0-416-gfe1d4d99 Python-3.12.4 torch-2.7.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.

In [10]:
import torch
import cv2
import matplotlib.pyplot as plt
import easyocr
import matplotlib.patches as patches

# -------------------------------
# 1. Load YOLOv5 model
# -------------------------------
model = torch.hub.load(
    'ultralytics/yolov5',
    'custom',
    path=r'C:\Users\user\Desktop\yolov5\yolov5\runs\train\exp5\weights\best.pt',
    force_reload=True
)

# -------------------------------
# 2. Initialize EasyOCR Reader
# -------------------------------
reader = easyocr.Reader(['en'], gpu=False)  # adjust gpu=True if you have a GPU

# -------------------------------
# 3. Define OCR Helper Functions
# -------------------------------
def extract_plate_text(plate_img):
    """
    Convert image to RGB (if needed) and extract text using EasyOCR.
    Returns the extracted text, raw OCR results, and the RGB image.
    """
    # Convert to RGB if the image is in BGR format
    if len(plate_img.shape) == 3:
        plate_rgb = cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB)
    else:
        plate_rgb = plate_img

    # Use EasyOCR to detect text in the cropped license plate image
    results = reader.readtext(plate_rgb)
    extracted_text = ' '.join([res[1] for res in results])
    return extracted_text, results, plate_rgb

def display_ocr_results(image, results):
    """
    Display OCR results with bounding boxes overlaid on the given image.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(image)

    for bbox, text, prob in results:
        # Extract points and create a rectangle patch for visualization
        top_left = tuple(map(int, bbox[0]))
        bottom_right = tuple(map(int, bbox[2]))
        rect_width = bottom_right[0] - top_left[0]
        rect_height = bottom_right[1] - top_left[1]
        rect = patches.Rectangle(top_left, rect_width, rect_height,
                                 linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text(top_left[0], top_left[1] - 10, text, color='blue', fontsize=12)
    plt.axis('off')
    plt.show()

# -------------------------------
# 4. Load and Process the Test Image
# -------------------------------
test_image_path = r'C:\Users\user\Desktop\testing images\new.jpg'  # adjust path
img = cv2.imread(test_image_path)
if img is None:
    raise Exception("Test image not found. Check the path.")

# Make a copy of the original image (used for cropping ROI)
orig_img = img.copy()

# Run YOLOv5 detection on the full image
results = model(img)
detections = results.xyxy[0].cpu().numpy()

# -------------------------------
# 5. Process Detected License Plate(s)
# -------------------------------
if len(detections) > 0:
    # Here we use the first detected license plate
    x_min, y_min, x_max, y_max, conf, cls = detections[0]
    x_min, y_min, x_max, y_max = map(int, (x_min, y_min, x_max, y_max))

    # Draw bounding box on the image for visualization
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Crop the detected license plate region from the original image
    plate_roi = orig_img[y_min:y_max, x_min:x_max]

    # Use EasyOCR to extract text from the cropped region
    plate_text, ocr_details, plate_rgb = extract_plate_text(plate_roi)
    print("Extracted License Plate Text:", plate_text.strip())

    # Display OCR results on the cropped ROI
    display_ocr_results(plate_rgb, ocr_details)

    # Optionally, display the full image with the detected bounding box
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected License Plate')
    plt.show()
else:
    print("No license plate detected.")


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\user/.cache\torch\hub\master.zip


YOLOv5  2025-5-14 Python-3.12.7 torch-2.7.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Using CPU. Note: This module is much faster with a GPU.
C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Extracted License Plate Text: "T838 ADH


In [9]:
import torch
import cv2
import easyocr
import numpy as np
import pickle
import os

class LicensePlateRecognizer:
    """
    A pipeline that loads a YOLOv5 trained model for license plate detection and uses EasyOCR to perform OCR on the detected plate region.
    """
    def __init__(self, weight_path, device='cpu'):
        """
        Initialize the recognizer by loading the YOLOv5 model and EasyOCR reader.
        
        :param weight_path: Path to the trained YOLOv5 weights file (e.g., best.pt)
        :param device: Device to run inference on ('cpu' or 'cuda')
        """
        self.device = device
        print("Loading YOLOv5 model...")
        self.model = torch.hub.load('ultralytics/yolov5', 'custom',
                                    path=weight_path, device=device, force_reload=True)
        print("Initializing EasyOCR Reader...")
        self.reader = easyocr.Reader(['en'], gpu=False)  # Set gpu=True if a GPU is available
    
    def detect_plate(self, img):
        """
        Run detection on an image to get the license plate region.
        
        :param img: Input image (NumPy array in BGR format).
        :return: plate_roi (the cropped region) and detections (raw YOLOv5 detection data).
        """
        results = self.model(img)
        detections = results.xyxy[0].cpu().numpy()
        if len(detections) == 0:
            return None, detections
        # For simplicity, select the first detection.
        x_min, y_min, x_max, y_max, conf, cls = detections[0]
        plate_roi = img[int(y_min):int(y_max), int(x_min):int(x_max)]
        return plate_roi, detections
    
    def recognize_plate(self, plate_roi):
        """
        Given a cropped license plate image, apply OCR using EasyOCR.
        
        :param plate_roi: Cropped image of the detected license plate.
        :return: Extracted text as a string.
        """
        # Convert ROI to RGB (EasyOCR expects RGB images)
        if len(plate_roi.shape) == 3:
            plate_rgb = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2RGB)
        else:
            plate_rgb = plate_roi
        
        # Run EasyOCR on the plate region and concatenate the detected text segments.
        results = self.reader.readtext(plate_rgb)
        extracted_text = ' '.join([res[1] for res in results])
        return extracted_text.strip()
    
    def predict_plate(self, image_path):
        """
        Load an image from disk, perform detection and OCR to identify the license plate text.
        
        :param image_path: Path to the test image.
        :return: A tuple of (plate_text, detections, original_image) where:
                 - plate_text is the OCR result (or None if no plate was detected),
                 - detections contains the YOLOv5 detection data,
                 - original_image is the loaded image.
        """
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image file not found: {image_path}")
        
        # Read the image.
        img = cv2.imread(image_path)
        if img is None:
            raise Exception("Failed to load the image; please check the format and path.")
        
        # Run detection.
        plate_roi, detections = self.detect_plate(img)
        if plate_roi is None:
            return None, detections, img
        
        # Get OCR result using EasyOCR.
        plate_text = self.recognize_plate(plate_roi)
        return plate_text, detections, img

# Example usage:
if __name__ == "__main__":
    # Define paths (adjust these paths accordingly)
    weight_file = r'C:\Users\user\Desktop\yolov5\yolov5\runs\train\exp5\weights\best.pt'
    test_image_file = r'C:\Users\user\Desktop\VOC-LICENSE-PLATE\JPEGImages\Cars1.png'
    
    # Initialize the recognizer.
    recognizer = LicensePlateRecognizer(weight_file, device='cpu')
    
    # Run the prediction.
    plate_text, detections, original_img = recognizer.predict_plate(test_image_file)
    
    if plate_text is not None:
        print("Detected License Plate Text:", plate_text)
    else:
        print("No license plate detected.")
    
    # (Optional) Save the entire pipeline object to disk using pickle.
    export_file = r'C:\Users\user\Desktop\license_plate_recognizer.pkl'
    with open(export_file, 'wb') as f:
        pickle.dump(recognizer, f)
    print(f"Pipeline exported to {export_file}")


Loading YOLOv5 model...
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\user/.cache\torch\hub\master.zip


YOLOv5  2025-5-14 Python-3.12.7 torch-2.7.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Using CPU. Note: This module is much faster with a GPU.


Initializing EasyOCR Reader...


C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Detected License Plate Text: PGoNN112
Pipeline exported to C:\Users\user\Desktop\license_plate_recognizer.pkl


In [22]:
# change directory
%cd C:/Users/user/Desktop/yolov5

# then run validation
!python val.py \
    --weights C:/Users/user/Desktop/yolov5/yolov5/runs/train/exp5/weights/best.pt \
    --data    C:/Users/user/Desktop/license_plate.yaml \
    --img     640 \
    --batch   16 \
    --iou     0.5


C:\Users\user\Desktop\yolov5


val: data=C:/Users/user/Desktop/license_plate.yaml, weights=['C:/Users/user/Desktop/yolov5/yolov5/runs/train/exp5/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.5, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v7.0-416-gfe1d4d99 Python-3.12.4 torch-2.7.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

val: Scanning C:\Users\user\Desktop\data\labels\val.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|##########| 87/87 [00:00<?, ?it/s]
val: Scanning C:\Users\user\Desktop\data\labels\val.cache... 87 images, 0 backgrounds, 0 corrupt: 100%|##########| 87/87 [00:00<?, ?it/s]

                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0%|          | 0/6 [00:00<?, ?it/s]
                 Class  

In [2]:
# Change to the correct yolov5 directory
%cd C:/Users/user/Desktop/yolov5/yolov5

# Export YOLOv5 model to TFLite
!python export.py --weights runs/train/exp5/weights/best.pt --img 640 --include tflite --int8

# Verify the TFLite model was created
import os
tflite_path = "C:/Users/user/Desktop/yolov5/yolov5/runs/train/exp5/weights/best.tflite"
if os.path.exists(tflite_path):
    print(f"TFLite model successfully created at {tflite_path}")
else:
    print("TFLite conversion failed. Check export.py logs.")

C:\Users\user\Desktop\yolov5\yolov5
Model: "functional_15"
â”Œâ”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¬â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¬â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¬â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”�
â”‚ Layer (type)        â”‚ Output Shape      â”‚    Param # â”‚ Connected to      â”‚
â”œâ”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¼â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¼â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¼â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¤
â”‚ input_layer_15      â”‚ (1, 640, 640, 3)  â”‚          0 â”‚ -                 â”‚
â”‚ (InputLayer)        â”‚                   â”‚            â”‚                   â”‚
â”œâ”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¼â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¼â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”¼â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€â”€

export: data=C:\Users\user\Desktop\yolov5\yolov5\data\coco128.yaml, weights=['runs/train/exp5/weights/best.pt'], imgsz=[640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=True, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5  v7.0-416-gfe1d4d99 Python-3.12.4 torch-2.7.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from runs\train\exp5\weights\best.pt with output shape (1, 25200, 6) (13.8 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...
WARNING  using Tensorflow 2.19.0 > 2.13.1 might cause issue when exporting the model to tflite https://github.com/ultralytics/yolov5/issues/12489

                 from  n    params  module                                  arguments      

In [10]:
import cv2
import numpy as np
import tensorflow.lite as tflite
import logging
import os

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('tflite_model_test.log'),
        logging.StreamHandler()
    ]
)

def preprocess_image(img, input_shape):
    """Preprocess image for TFLite model."""
    try:
        # Resize to model input size (e.g., 640x640)
        img_resized = cv2.resize(img, input_shape)
        # Convert to RGB
        img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
        # Normalize to [0, 1] and convert to UINT8
        img_array = (img_rgb / 255.0 * 255).astype(np.uint8)
        img_array = np.expand_dims(img_array, axis=0)
        # Save preprocessed image for inspection
        cv2.imwrite('preprocessed_image.jpg', img_rgb)
        logging.info("Preprocessed image saved to preprocessed_image.jpg")
        return img_array
    except Exception as e:
        logging.error(f"Image preprocessing failed: {str(e)}")
        raise

def postprocess_detections(output, conf_thres=0.3):
    """Process single TFLite output tensor to extract bounding boxes."""
    try:
        detections = output[0]  # Shape: (num_detections, 6)
        filtered_detections = []
        logging.info(f"Total detections: {len(detections)}")
        # Log first 10 detections for debugging
        for i, det in enumerate(detections[:10]):
            y_min, x_min, y_max, x_max, score, cls = det
            logging.info(f"Detection {i}: y_min={y_min:.2f}, x_min={x_min:.2f}, y_max={y_max:.2f}, x_max={x_max:.2f}, score={score:.2f}, class={cls}")
        # Filter detections by confidence
        for det in detections:
            y_min, x_min, y_max, x_max, score, cls = det
            if score > conf_thres:
                filtered_detections.append([x_min, y_min, x_max, y_max, score, cls])
        logging.info(f"Filtered detections (score > {conf_thres}): {len(filtered_detections)}")
        return np.array(filtered_detections)
    except Exception as e:
        logging.error(f"Detection postprocessing failed: {str(e)}")
        raise

def test_model(model_path, image_path):
    """Test TFLite model on a sample image."""
    try:
        # Verify files exist
        if not os.path.isfile(model_path):
            logging.error(f"Model file not found: {model_path}")
            return
        if not os.path.isfile(image_path):
            logging.error(f"Image file not found: {image_path}")
            return

        # Load TFLite model
        logging.info("Loading TFLite model...")
        interpreter = tflite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        logging.info(f"Input details: {input_details}")
        logging.info(f"Output details: {output_details}")

        # Load and preprocess image
        img = cv2.imread(image_path)
        if img is None:
            logging.error("Failed to load image")
            return
        logging.info(f"Image loaded: {image_path}, shape={img.shape}")
        input_shape = (input_details[0]['shape'][2], input_details[0]['shape'][1])
        input_data = preprocess_image(img, input_shape)

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logging.info(f"Output shape: {output.shape}")

        # Process detections
        detections = postprocess_detections(output)

        # Visualize results
        if len(detections) > 0:
            logging.info("Detections found!")
            for i, (x_min, y_min, x_max, y_max, conf, cls) in enumerate(detections):
                h, w = img.shape[:2]
                x_min, x_max = int(x_min * w), int(x_max * w)
                y_min, y_max = int(y_min * h), int(y_max * h)
                if x_min < x_max and y_min < y_max:
                    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    cv2.putText(img, f"Conf: {conf:.2f}", (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                logging.info(f"Detection {i}: x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}, conf={conf:.2f}")
            cv2.imwrite('output_test.jpg', img)
            logging.info("Output image with bounding boxes saved to output_test.jpg")
        else:
            logging.info("No detections found.")
            cv2.imwrite('output_test.jpg', img)
            logging.info("Original image saved to output_test.jpg")

    except Exception as e:
        logging.error(f"Test failed: {str(e)}")

if __name__ == "__main__":
    model_path = r"C:\Users\user\Desktop\yolov5\yolov5\runs\train\exp5\weights\best-int8.tflite"
    image_path = r"C:\Users\user\Desktop\VOC-LICENSE-PLATE\JPEGImages\Cars1.png"
    test_model(model_path, image_path)

2025-05-25 17:25:29,316 - INFO - Loading TFLite model...
2025-05-25 17:25:29,479 - INFO - Input details: [{'name': 'serving_default_keras_tensor_121:0', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921568859368563, 0), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
2025-05-25 17:25:29,479 - INFO - Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 405, 'shape': array([    1, 25200,     6], dtype=int32), 'shape_signature': array([    1, 25200,     6], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00868131686002016, 1), 'quantization_parameters': {'scales': array([0.00868132], dtype=float32), 'zero_points': array([1], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
2025-05-25 17:25:2

In [12]:
import cv2
import numpy as np
import tensorflow.lite as tflite
import logging
import os

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('tflite_model_test.log'),
        logging.StreamHandler()
    ]
)

def preprocess_image(img, input_shape):
    """Preprocess image for TFLite model."""
    try:
        img_resized = cv2.resize(img, input_shape)
        img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
        img_array = (img_rgb / 255.0 * 255).astype(np.uint8)
        img_array = np.expand_dims(img_array, axis=0)
        cv2.imwrite('preprocessed_image.jpg', img_rgb)
        logging.info("Preprocessed image saved to preprocessed_image.jpg")
        return img_array
    except Exception as e:
        logging.error(f"Image preprocessing failed: {str(e)}")
        raise

def dequantize_output(output, scale, zero_point):
    """Dequantize uint8 output to float."""
    return (output.astype(np.float32) - zero_point) * scale

def postprocess_detections(output, scale, zero_point, conf_thres=0.1):
    """Process single TFLite output tensor to extract bounding boxes."""
    try:
        # Dequantize output
        detections = dequantize_output(output[0], scale, zero_point)  # Shape: (num_detections, 6)
        filtered_detections = []
        logging.info(f"Total detections: {len(detections)}")
        # Log first 10 detections
        for i, det in enumerate(detections[:10]):
            y_min, x_min, y_max, x_max, score, cls = det
            logging.info(f"Detection {i}: y_min={y_min:.4f}, x_min={x_min:.4f}, y_max={y_max:.4f}, x_max={x_max:.4f}, score={score:.4f}, class={cls:.0f}")
        # Filter detections
        for det in detections:
            y_min, x_min, y_max, x_max, score, cls = det
            if score > conf_thres and cls == 0:  # Assume class 0 is license plate
                filtered_detections.append([x_min, y_min, x_max, y_max, score, cls])
        logging.info(f"Filtered detections (score > {conf_thres}, class=0): {len(filtered_detections)}")
        return np.array(filtered_detections)
    except Exception as e:
        logging.error(f"Detection postprocessing failed: {str(e)}")
        raise

def test_model(model_path, image_path):
    """Test TFLite model on a sample image."""
    try:
        if not os.path.isfile(model_path):
            logging.error(f"Model file not found: {model_path}")
            return
        if not os.path.isfile(image_path):
            logging.error(f"Image file not found: {image_path}")
            return

        # Load TFLite model
        logging.info("Loading TFLite model...")
        interpreter = tflite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        logging.info(f"Input details: {input_details}")
        logging.info(f"Output details: {output_details}")

        # Get quantization parameters
        scale = output_details[0]['quantization_parameters']['scales'][0]
        zero_point = output_details[0]['quantization_parameters']['zero_points'][0]
        logging.info(f"Output quantization: scale={scale}, zero_point={zero_point}")

        # Load and preprocess image
        img = cv2.imread(image_path)
        if img is None:
            logging.error("Failed to load image")
            return
        logging.info(f"Image loaded: {image_path}, shape={img.shape}")
        input_shape = (input_details[0]['shape'][2], input_details[0]['shape'][1])
        input_data = preprocess_image(img, input_shape)

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logging.info(f"Output shape: {output.shape}")

        # Process detections
        detections = postprocess_detections(output, scale, zero_point)

        # Visualize results
        if len(detections) > 0:
            logging.info("Detections found!")
            for i, (x_min, y_min, x_max, y_max, conf, cls) in enumerate(detections):
                h, w = img.shape[:2]
                x_min, x_max = int(x_min * w), int(x_max * w)
                y_min, y_max = int(y_min * h), int(y_max * h)
                # Validate bounding box
                if x_min < x_max and y_min < y_max and 0 <= x_min < w and 0 <= y_min < h:
                    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    cv2.putText(img, f"Conf: {conf:.2f}", (x_min, y_min - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    logging.info(f"Detection {i}: x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}, conf={conf:.2f}")
                else:
                    logging.warning(f"Invalid bounding box for detection {i}")
            cv2.imwrite('output_test.jpg', img)
            logging.info("Output image with bounding boxes saved to output_test.jpg")
        else:
            logging.info("No detections found.")
            cv2.imwrite('output_test.jpg', img)
            logging.info("Original image saved to output_test.jpg")

    except Exception as e:
        logging.error(f"Test failed: {str(e)}")

if __name__ == "__main__":
    model_path = r"C:\Users\user\Desktop\yolov5\yolov5\runs\train\exp5\weights\best-int8.tflite"
    image_path = r"C:\Users\user\Desktop\VOC-LICENSE-PLATE\JPEGImages\Cars2.png"
    test_model(model_path, image_path)


2025-05-25 17:35:48,663 - INFO - Loading TFLite model...
2025-05-25 17:35:48,765 - INFO - Input details: [{'name': 'serving_default_keras_tensor_121:0', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.003921568859368563, 0), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
2025-05-25 17:35:48,768 - INFO - Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 405, 'shape': array([    1, 25200,     6], dtype=int32), 'shape_signature': array([    1, 25200,     6], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.00868131686002016, 1), 'quantization_parameters': {'scales': array([0.00868132], dtype=float32), 'zero_points': array([1], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
2025-05-25 17:35:4